<a href="https://colab.research.google.com/github/Zarif123/SSLM-Project/blob/main/chess_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install python-chess

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
folder = "/content/gdrive/MyDrive/Statistical_LM_Group_Folder"

csv_file = "chess_data.csv"
csv_path = f"{folder}/{csv_file}"

model_file = "model.pth"
model_path = f"{folder}/{model_file}"

Mounted at /content/gdrive


In [3]:
from transformers import AutoTokenizer, BertModel, BertConfig
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train_loop(model, linear, optimizer, tokenizer, train, classes, epochs):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1} ///////////////////////////////")

        train_len = len(train)
        total_loss = 0.0

        for train_i in range(train_len):
            observation = train[train_i]
            moves = []
            labels = []
            mask = torch.zeros((classes, 2), dtype=float).to(device)

            for class_i in range(classes):
                move_order = observation[class_i][0]
                elo_bucket = observation[class_i][1]
                moves.append(move_order)
                labels.append(elo_bucket)
                mask[class_i][elo_bucket] = 1
            
            inputs = tokenizer(moves, max_length=256, padding="max_length", truncation=True, return_tensors="pt")
            # inputs = inputs.to(device)
            inputs['input_ids'] = inputs['input_ids'].to(device)
            inputs['token_type_ids'] = inputs['token_type_ids'].to(device)
            inputs['attention_mask'] = inputs['attention_mask'].to(device)

            optimizer.zero_grad()
            hidden = model(**inputs)

            logits = torch.matmul(hidden.last_hidden_state[:, 0, :], linear)
            probs = F.softmax(logits, dim=1)
            correct_probs = probs * mask
            log_probs = torch.log(torch.sum(correct_probs, dim=1)).squeeze()
            loss = -torch.sum(log_probs)

            total_loss += loss.item()

            loss.backward()
            optimizer.step()
            
            if train_i % 1000 == 0:
                print(train_i, "/", train_len)
        
        average_loss = total_loss / train_len
        print(f"Average Loss: {average_loss}")

        torch.save(model.state_dict(), model_path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
optimizer = optim.Adam(model.parameters(), lr=3e-5)
linear = torch.rand(768,2)

In [6]:
model = model.to(device)
linear = linear.to(device)